In [1]:
import mne
import matplotlib.pyplot as plt
import pre_process as pp
import utils as u
import pandas as pd
import time
import joblib as jb
from tqdm import tqdm
import os

AttributeError: module 'mne' has no attribute '__version__'

In [8]:
path_list = u.get_path_list('/u/pmihelj/isip.piconepress.com/projects/nedc/data/tuh_eeg/tuh_eeg_abnormal/v3.0.1/edf',['.edf'],True)

In [5]:
print(type(path_list))
print(path_list.shape)

<class 'numpy.ndarray'>
(2993,)


In [8]:
path_id_list= [f"campione_{i}" for i in range(1, 2994)]

In [11]:
df_path = pd.DataFrame({'Path_id':path_id_list, 'Path':path_list})

In [14]:
df_path.tail()

,Path_id,Path
2988,campione_2989,/u/pmihelj/isip.piconepress.com/projects/nedc/...
2989,campione_2990,/u/pmihelj/isip.piconepress.com/projects/nedc/...
2990,campione_2991,/u/pmihelj/isip.piconepress.com/projects/nedc/...
2991,campione_2992,/u/pmihelj/isip.piconepress.com/projects/nedc/...
2992,campione_2993,/u/pmihelj/isip.piconepress.com/projects/nedc/...


In [9]:
df_path.to_csv('path.csv', index=False, encoding='utf-8-sig')

In [9]:
def worker(in_file, out_dir, out_prefix):
    #try:
    dg = pp.data_gen(in_file, out_dir, out_prefix)
    dg.save_final_data(seg_len=1.0, merge_len=0.2)
    #except:
    #    flag = False
    #    time.sleep(3.0)
    #else:
    #    flag = True
    return in_file#, flag

In [10]:
print(os.sched_getaffinity(0))  # core accessibili al processo
print(len(os.sched_getaffinity(0)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
16


In [11]:
md_file = "path.csv"
out_dir = "./raw_data"
log_file = "./raw_data/log.csv"

In [12]:
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

df_path = pd.read_csv(md_file)
tasks = [(Pt, Pt_id) for Pt, Pt_id in zip(df_path.Path, df_path.Path_id)]
tasks[0]

('/u/pmihelj/isip.piconepress.com/projects/nedc/data/tuh_eeg/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaabdo_s003_t000.edf',
 'campione_1')

In [2]:
n_jobs = jb.cpu_count()
res = jb.Parallel(n_jobs=n_jobs)(jb.delayed(worker)(in_file, out_dir, out_f_id) for in_file, out_f_id in tqdm(tasks))

NameError: name 'jb' is not defined

In [ ]:
df_log = pd.DataFrame(res, columns=['path', 'status'])
df_log.to_csv(log_file, index=False, encoding="utf-8-sig") 